In [93]:
%matplotlib inline

In [94]:
from __future__ import division

import re
import codecs

import numpy as np
import matplotlib.pyplot as plt

from sqlearn.crfsuite import crfutils
from sqlearn.crfsuite import ner

import MeCab
mecab = MeCab.Tagger()

In [101]:
raw = u"""山形ふれあいマーケット
ページID ： 104890
５月から１１月まで、仙台市にて「山形ふれあいマーケット」を開催しています。
とれたての野菜・果物など旬の美味しいものや、牛串や漬物、手作り味噌なども販売し、
山形市の魅力を発信します。
とき	2016年5月25日 ～ 11月10日
ところ	仙台市匂当台公園　市民広場またはグリーンハウス前
対象・定員	どなたでもお越しください
費用	 
申し込み詳細	 
関連リンク"""

In [102]:
data = []
for line in raw.split('\n'):
    encoded_line = line.encode('utf-8')
    node = mecab.parseToNode(encoded_line)
    node = node.next
    while node:
        surface = node.surface.decode('utf-8')
        features = node.feature.decode('utf-8').split(',')
        data.append('%s %s_%s_%s' % (surface, features[0], features[1], features[2]))
        node = node.next
print len(data)
data = '\n'.join(data)

111


In [103]:
print data

山形 名詞_地名_*
ふれあい 動詞_*_子音動詞ワ行
マーケット 名詞_普通名詞_*
 BOS/EOS_*_*
ページ 名詞_普通名詞_*
ID 名詞_組織名_*
： 特殊_記号_*
104890 名詞_数詞_*
 BOS/EOS_*_*
５ 名詞_数詞_*
月 接尾辞_名詞性名詞助数辞_*
から 助詞_格助詞_*
１１ 名詞_数詞_*
月 接尾辞_名詞性名詞助数辞_*
まで 助詞_格助詞_*
、 特殊_読点_*
仙台 名詞_地名_*
市 接尾辞_名詞性特殊接尾辞_*
にて 助詞_格助詞_*
「 特殊_括弧始_*
山形 名詞_地名_*
ふれあい 動詞_*_子音動詞ワ行
マーケット 名詞_普通名詞_*
」 特殊_括弧終_*
を 助詞_格助詞_*
開催 名詞_サ変名詞_*
して 動詞_*_サ変動詞
い 接尾辞_動詞性接尾辞_母音動詞
ます 接尾辞_動詞性接尾辞_動詞性接尾辞ます型
。 特殊_句点_*
 BOS/EOS_*_*
とれ 動詞_*_母音動詞
たて 接尾辞_名詞性述語接尾辞_*
の 助詞_接続助詞_*
野菜 名詞_普通名詞_*
・ 特殊_記号_*
果物 名詞_普通名詞_*
など 助詞_副助詞_*
旬 名詞_普通名詞_*
の 助詞_格助詞_*
美味しい 形容詞_*_イ形容詞イ段
もの 名詞_形式名詞_*
や 助詞_接続助詞_*
、 特殊_読点_*
牛 名詞_普通名詞_*
串 名詞_普通名詞_*
や 助詞_接続助詞_*
漬物 名詞_普通名詞_*
、 特殊_読点_*
手作り 名詞_サ変名詞_*
味噌 名詞_普通名詞_*
など 助詞_副助詞_*
も 助詞_副助詞_*
販売 名詞_サ変名詞_*
し 動詞_*_サ変動詞
、 特殊_読点_*
 BOS/EOS_*_*
山形 名詞_地名_*
市 接尾辞_名詞性特殊接尾辞_*
の 助詞_接続助詞_*
魅力 名詞_普通名詞_*
を 助詞_格助詞_*
発信 名詞_サ変名詞_*
し 動詞_*_サ変動詞
ます 接尾辞_動詞性接尾辞_動詞性接尾辞ます型
。 特殊_句点_*
 BOS/EOS_*_*
とき 名詞_普通名詞_*
2016 名詞_数詞_*
年 接尾辞_名詞性名詞助数辞_*
5 名詞_数詞_*
月 接尾辞_名詞性名詞助数辞_*
25 名詞_数詞_*
日 接尾辞_名詞性名詞助数辞_*
～ 特殊_記号_*
11 名詞_数詞_*
月 接

In [104]:
X, sent = [], []
for line in data.split('\n'):
    splited_line = line.split(' ')
    sent.append('%s %s' % (splited_line[0], splited_line[1]))

sent.append('\n')
for item in crfutils.readiter(sent, ['w', 'pos'], ' '):
    ner.feature_extractor(item)
    X.append(item)
        
X = [[feature['F'] for feature in sent] for sent in X]

X = np.asarray(X)
print len(X[0])

111


In [105]:
print X[0][0]
print X[0][1]

[u'w[0]=\u5c71\u5f62', u'w[1]=\u3075\u308c\u3042\u3044', u'w[2]=\u30de\u30fc\u30b1\u30c3\u30c8', u'w[3]=', u'w[4]=\u30da\u30fc\u30b8', u'w[0]|w[1]=\u5c71\u5f62|\u3075\u308c\u3042\u3044', u'w[1]|w[2]=\u3075\u308c\u3042\u3044|\u30de\u30fc\u30b1\u30c3\u30c8', u'w[2]|w[3]=\u30de\u30fc\u30b1\u30c3\u30c8|', u'w[3]|w[4]=|\u30da\u30fc\u30b8', u'pos[0]=\u540d\u8a5e_\u5730\u540d_*', u'pos[1]=\u52d5\u8a5e_*_\u5b50\u97f3\u52d5\u8a5e\u30ef\u884c', u'pos[2]=\u540d\u8a5e_\u666e\u901a\u540d\u8a5e_*', u'pos[3]=BOS/EOS_*_*', u'pos[4]=\u540d\u8a5e_\u666e\u901a\u540d\u8a5e_*', u'pos[0]|pos[1]=\u540d\u8a5e_\u5730\u540d_*|\u52d5\u8a5e_*_\u5b50\u97f3\u52d5\u8a5e\u30ef\u884c', u'pos[1]|pos[2]=\u52d5\u8a5e_*_\u5b50\u97f3\u52d5\u8a5e\u30ef\u884c|\u540d\u8a5e_\u666e\u901a\u540d\u8a5e_*', u'pos[2]|pos[3]=\u540d\u8a5e_\u666e\u901a\u540d\u8a5e_*|BOS/EOS_*_*', u'pos[3]|pos[4]=BOS/EOS_*_*|\u540d\u8a5e_\u666e\u901a\u540d\u8a5e_*', u'pos[0]|pos[1]|pos[2]=\u540d\u8a5e_\u5730\u540d_*|\u52d5\u8a5e_*_\u5b50\u97f3\u52d5\u8a

In [106]:
import pycrfsuite as crf

tagger = crf.Tagger()
tagger.open('model')
y = tagger.tag(X[0])

for e in zip(data.split('\n'), y):
    print e[0].split()[0], e[1]

山形 B-OCCATION
ふれあい I-OCCATION
マーケット I-OCCATION
BOS/EOS_*_* O
ページ O
ID O
： O
104890 O
BOS/EOS_*_* O
５ O
月 O
から O
１１ O
月 O
まで O
、 O
仙台 O
市 O
にて O
「 O
山形 O
ふれあい O
マーケット O
」 O
を O
開催 O
して O
い O
ます O
。 O
BOS/EOS_*_* O
とれ O
たて O
の O
野菜 O
・ O
果物 O
など O
旬 O
の O
美味しい O
もの O
や O
、 O
牛 O
串 O
や O
漬物 O
、 O
手作り O
味噌 O
など O
も O
販売 O
し O
、 O
BOS/EOS_*_* O
山形 O
市 O
の O
魅力 O
を O
発信 O
し O
ます O
。 O
BOS/EOS_*_* O
とき O
2016 O
年 O
5 O
月 O
25 O
日 O
～ O
11 O
月 O
10 O
日 O
BOS/EOS_*_* O
ところ O
仙台 O
市 O
匂 O
当 O
台 O
公園 O
特殊_空白_* O
市民 O
広場 O
または O
グリーンハウス O
前 O
BOS/EOS_*_* O
対象 O
・ O
定員 O
どなた O
でも O
お O
越し O
ください O
BOS/EOS_*_* O
費用 O
BOS/EOS_*_* O
申し込み O
詳細 O
BOS/EOS_*_* O
関連 O
リンク O
BOS/EOS_*_* O
